<a href="https://colab.research.google.com/github/yeonmi02/machine-learning-practice/blob/main/250416_cross_validation_hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 교차 검증과 그리드 서치

## 검증 데이터셋

In [16]:
# 데이터 셋 불러오기
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine-date')

### 문제 1 : wine 데이터 확인

In [17]:
# wine 처음 5개 행 데이터 확인
wine.head(5)

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [21]:
# wine 전체 행과 열의 개수 확인
print(wine.shape)

(6497, 4)


In [8]:
# wine 데이터 통계값 확인 (각 특성별 평균, 표준편차, 최소값, 최대값 등)
wine.describe()

,alcohol,sugar,pH,class
count,6497.000000,6497.000000,6497.000000,6497.000000
mean,10.491801,5.443235,3.218501,0.753886
std,1.192712,4.757804,0.160787,0.430779
min,8.000000,0.600000,2.720000,0.000000
25%,9.500000,1.800000,3.110000,1.000000
50%,10.300000,3.000000,3.210000,1.000000
75%,11.300000,8.100000,3.320000,1.000000
max,14.900000,65.800000,4.010000,1.000000


In [9]:
# 화이트 와인, 레드 와인 데이터 개수 확인
wine['class'].value_counts()

,count
class,
1.0,4898
0.0,1599


### 데이터셋 분류

In [24]:
# 배열로 설정하기
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [25]:
# 데이터 셋 분류하기
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [27]:
# 검증 데이터 셋 나누기
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [28]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [30]:
# 결정트리 학습 시키기
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

# 과적합 결과 확인 가능
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


## 교차 검증

In [31]:
# 교차 검증 사용하기
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00829673, 0.00799894, 0.00820446, 0.00812912, 0.0078733 ]), 'score_time': array([0.00115776, 0.00104117, 0.00106835, 0.00096393, 0.00105429]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [32]:
import numpy as np
# 평균을 구하기
print(np.mean(scores['test_score']))

0.855300214703487


In [33]:
# 분할기 넣기
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [34]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


## 하이퍼파라미터 튜닝

In [35]:
from sklearn.model_selection import GridSearchCV
# 하이퍼파라미터 정의
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [38]:
# 그리드 서치
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [39]:
# 그리드 서치 학습
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [41]:
# 최적의 모델
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [43]:
# 최적의 파라미터 값
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [44]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [45]:
# 가장 최대값을 갖는 인덱스를 반환해준다
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [48]:
# 넘파이를 통해서 정의하는 방법
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

In [47]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [49]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': np.float64(0.0004), 'min_samples_split': 12}


In [50]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [56]:
# 교차검증 수행 시간 프린트
gs.cv_results_['mean_fit_time']

array([0.00696659, 0.00655069, 0.00655575, ..., 0.00664797, 0.00500259,
       0.00544701])

### 랜덤 서치

In [57]:
from scipy.stats import uniform, randint

In [58]:
# 균등 분포 샘플링
rgen = randint(0, 10)
rgen.rvs(10)

array([0, 2, 9, 1, 5, 8, 2, 0, 8, 3])

In [59]:
np.unique(rgen.rvs(1000), return_counts=True) # 빈도도 함께 출력

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([107,  91,  96,  99, 104,  98,  94, 111, 104,  96]))

In [63]:
# 0과 1에서 랜덤으로 뽑는 것
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.74077806, 0.7489022 , 0.76305291, 0.52293371, 0.57895382,
       0.25664035, 0.70525986, 0.68143421, 0.84639061, 0.57032023])

In [64]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [65]:
from sklearn.model_selection import RandomizedSearchCV

rs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
rs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x789f876aa7d0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x789f876a16d0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x789f89162290>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x789f876a2cd0>},
                   random_state=42)

In [66]:
print(rs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': np.float64(0.00034102546602601173), 'min_samples_leaf': 7, 'min_samples_split': 13}


In [67]:
print(np.max(rs.cv_results_['mean_test_score']))

0.8695428296438884


In [68]:
dt = rs.best_estimator_

print(dt.score(test_input, test_target))

0.86


In [69]:
rs.cv_results_['mean_fit_time']

array([0.00705619, 0.00693936, 0.0073257 , 0.00847707, 0.008781  ,
       0.00936418, 0.00669041, 0.00675716, 0.0070683 , 0.00688043,
       0.00694237, 0.00710478, 0.006776  , 0.00722518, 0.00679574,
       0.00708795, 0.0066411 , 0.00696039, 0.00700207, 0.01108508,
       0.00878377, 0.00664163, 0.0068141 , 0.00690122, 0.00659328,
       0.00779886, 0.00814276, 0.00688887, 0.00728178, 0.00758557,
       0.0065166 , 0.00709872, 0.00657759, 0.00867987, 0.00824986,
       0.00688601, 0.00662808, 0.00727439, 0.0065671 , 0.00778785,
       0.00962481, 0.00808096, 0.00720544, 0.00775619, 0.00698895,
       0.00675316, 0.00662127, 0.00721903, 0.00855079, 0.00693793,
       0.00763049, 0.00757942, 0.00871472, 0.00646968, 0.00668464,
       0.00698543, 0.00660462, 0.00742025, 0.00767851, 0.00661106,
       0.00780368, 0.00670543, 0.00645723, 0.00663939, 0.00636835,
       0.00706797, 0.00654674, 0.00691147, 0.00661602, 0.00662794,
       0.00652161, 0.00875478, 0.00691013, 0.00704846, 0.00677

In [70]:
print(np.mean(rs.cv_results_['mean_fit_time']))

0.0073068032264709485


### 결정트리 분할 옵션 변경

In [76]:
# 결정트리의 옵션을 변경
rs2 = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
rs2.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x789f876aa7d0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x789f876a16d0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x789f89162290>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x789f876a2cd0>},
                   random_state=42)

In [73]:
print(rs2.best_params_)
print(np.max(rs2.cv_results_['mean_test_score']))

dt = rs2.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': np.float64(0.00011407982271508446), 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077


In [74]:
rs2.cv_results_['mean_fit_time']

array([0.00723181, 0.00348587, 0.00919237, 0.00734015, 0.00822854,
       0.00608344, 0.00968633, 0.00613627, 0.00947609, 0.00433869,
       0.00517974, 0.00735345, 0.00353384, 0.00324764, 0.00301628,
       0.0060884 , 0.00828424, 0.00562963, 0.00753188, 0.00316343,
       0.00406551, 0.00310988, 0.00496969, 0.00305257, 0.00376806,
       0.00815639, 0.00454283, 0.00334716, 0.00304847, 0.00376897,
       0.00287728, 0.00318036, 0.00278955, 0.00503001, 0.00694723,
       0.0079566 , 0.00302005, 0.00494256, 0.00427895, 0.00589528,
       0.00849476, 0.00460138, 0.00590634, 0.00312018, 0.00310102,
       0.00318255, 0.00363965, 0.00708737, 0.00598898, 0.00484223,
       0.00976753, 0.00306468, 0.00304928, 0.00290856, 0.00312486,
       0.00437207, 0.006074  , 0.00593648, 0.00667276, 0.00511599,
       0.00333896, 0.01067181, 0.00344605, 0.00611887, 0.00745134,
       0.00689673, 0.00436492, 0.01129541, 0.01498556, 0.00547748,
       0.00298567, 0.00720472, 0.01092324, 0.00898361, 0.00630

In [75]:
print(np.mean(rs2.cv_results_['mean_fit_time']))

0.005208634376525879


문제 2 : 위 코드가 기존 랜덤 서치 코드와 다른 점을 2가지 적어보세요.

- 모델에 대해서 정의를 해줄 때 기존에는 최적의 값을 분할했지만 위 코드는 랜덤하게 분할한다.

- 기존 랜덤 서치 코드보다 위 코드가 속도가 더 빠르다